In [3]:
import sys
sys.path.insert(0,'/Users/chemon/ARGO_NEW/NEW_LOCODOX/locodox_python/source')
import argopy
import xarray as xr
from m_argo_data import open_argo_multi_profile_file, get_argo_launch_date, get_argo_optode_height, get_argo_data_for_WOA

In [6]:
ds_argo_meta = open_argo_multi_profile_file('6990661','/Volumes/argo/gdac/dac/coriolis/','meta')

/Volumes/argo/gdac/dac/coriolis/6990661/6990661_meta.nc


In [7]:
ds_argo_meta


<xarray.Dataset> Size: 821kB
Dimensions:                               (N_TRANS_SYSTEM: 1,
                                           N_POSITIONING_SYSTEM: 2,
                                           N_LAUNCH_CONFIG_PARAM: 55,
                                           N_CONFIG_PARAM: 7, N_MISSIONS: 3,
                                           N_SENSOR: 6, N_PARAM: 15)
Dimensions without coordinates: N_TRANS_SYSTEM, N_POSITIONING_SYSTEM,
                                N_LAUNCH_CONFIG_PARAM, N_CONFIG_PARAM,
                                N_MISSIONS, N_SENSOR, N_PARAM
Data variables: (12/65)
    DATA_TYPE                             <U16 64B ...
    FORMAT_VERSION                        <U4 16B ...
    HANDBOOK_VERSION                      <U4 16B ...
    DATE_CREATION                         datetime64[ns] 8B ...
    DATE_UPDATE                           datetime64[ns] 8B ...
    PLATFORM_NUMBER                       int64 8B ...
    ...                                    ...
    PARAMETER_UNITS                       (N_PARAM) <U32 2kB ...
    PARAMETER_ACCURACY                    (N_PARAM) <U32 2kB ...
    PARAMETER_RESOLUTION                  (N_PARAM) <U32 2kB ...
    PREDEPLOYMENT_CALIB_EQUATION          (N_PARAM) <U4096 246kB ...
    PREDEPLOYMENT_CALIB_COEFFICIENT       (N_PARAM) <U4096 246kB ...
    PREDEPLOYMENT_CALIB_COMMENT           (N_PARAM) <U4096 246kB ...
Attributes:
    title:                Argo float metadata file
    institution:          CORIOLIS
    source:               Argo float
    history:              2025-10-22T08:27:24Z creation; 2025-10-22T08:27:24Z...
    references:           http://www.argodatamgt.org/Documentation
    user_manual_version:  3.1
    Conventions:          Argo-3.1 CF-1.6
    decoder_version:      CODA_080j
    id:                   https://doi.org/10.17882/42182

In [20]:
ds_argo_meta["PARAMETER"].str.strip()

<xarray.DataArray 'PARAMETER' (N_PARAM: 15)> Size: 1kB
array(['MTIME', 'PRES', 'TEMP', 'PSAL', 'C1PHASE_DOXY', 'C2PHASE_DOXY',
       'TEMP_DOXY', 'DOXY', 'PPOX_DOXY', 'COUNT_DOXY',
       'LED_FLASHING_COUNT_DOXY', 'TEMP_COUNT_DOXY', 'TEMP_DOXY_2',
       'DOXY_2', 'PPOX_DOXY_2'], dtype='<U23')
Dimensions without coordinates: N_PARAM

In [21]:
mask = ds_argo_meta["PARAMETER"].str.strip() == "DOXY"

coef_doxy = ds_argo_meta["PREDEPLOYMENT_CALIB_COEFFICIENT"].where(mask, drop=True)

In [22]:
coef_doxy

<xarray.DataArray 'PREDEPLOYMENT_CALIB_COEFFICIENT' (N_PARAM: 1)> Size: 8B
array(['Spreset=0.000000e+00; Pcoef1=1.000000e-01, Pcoef2=2.200000e-04, Pcoef3=4.190000e-02; B0=-6.245230e-03, B1=-7.376140e-03, B2=-1.034100e-02, B3=-8.170830e-03; C0=-4.886820e-07; PhaseCoef0=-6.990000e-01, PhaseCoef1=1.000000e+00, PhaseCoef2=0.000000e+00, PhaseCoef3=0.000000e+00; c0=3.320880e-03, c1=1.452830e-04, c2=2.413870e-06, c3=1.435830e+02, c4=-1.694500e-01, c5=-3.202980e+01, c6=2.775230e+00; D0=2.445430e+01, D1=-6.745090e+01, D2=-4.848900e+00, D3=-5.440000e-04                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [34]:
info_coef_doxy = coef_doxy.item().lower()

In [39]:
import re # expression reguliere
pcoef2 = float(re.search(r"pcoef2=([\deE\+\-\.]+)", info_coef_doxy).group(1))
pcoef3 = float(re.search(r"pcoef3=([\deE\+\-\.]+)", info_coef_doxy).group(1))

print(pcoef2)
print(pcoef3)
print(type(pcoef2))

0.00022
0.0419
<class 'float'>


In [44]:
def get_coef_value(s, key):
    """Retourne la valeur float associée à une clé dans une chaîne s."""
    chaine_ok = re.search(rf"{key.lower()}=([\deE\+\-\.]+)", s)
    return float(chaine_ok.group(1)) if chaine_ok else None

pcoef2 = get_coef_value(info_coef_doxy, 'Pcoef2')
print(pcoef2,type(pcoef2))
pcoef3 = get_coef_value(info_coef_doxy, 'Pcoef3')
print(pcoef3)


0.00022 <class 'float'>
0.0419
